In [42]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from sklearn.ensemble import IsolationForest
from scipy.stats import zscore

import seaborn as sns
import matplotlib.pyplot as plt

# import warnings
# warnings.filterwarnings("ignore")

In [89]:
training=pd.read_csv('training_data.csv', sep=',')
test=pd.read_csv('songs_to_classify.csv', sep=',')
training.sample(5)

,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,label
424,0.00987,0.528,210173,0.672,0.000704,8,0.141,-6.540,1,0.0316,116.078,4,0.467,1
690,0.00422,0.792,202963,0.454,0.000000,9,0.285,-6.753,1,0.4580,161.993,4,0.674,0
86,0.01020,0.511,174893,0.777,0.015500,11,0.215,-9.538,0,0.0380,140.000,4,0.237,1
259,0.10000,0.722,98595,0.836,0.000021,10,0.384,-7.231,0,0.0837,125.064,4,0.914,1
620,0.99400,0.351,124760,0.137,0.064600,10,0.171,-29.601,1,0.0693,78.375,4,0.543,1


In [95]:
X, y = training.iloc[:, 0:12], training.iloc[:, 13]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
cols = list(map(lambda x: "red" if x == 0 else "blue", training.iloc[:,13]))
feature_selection = range(0, 12) # change here to study other features
pd.plotting.scatter_matrix(training.iloc[:,feature_selection], c=cols, figsize=(12, 12));

In [45]:
# Q1 = training.quantile(0.25)
# Q3 = training.quantile(0.75)
# IQR = Q3 - Q1

# criteria = 1.7
# outlier_condition = (training < (Q1 - criteria * IQR)) | (training > (Q3 + criteria * IQR))

# mask = ~outlier_condition.any(axis=1)
# training = training[mask]

# cols = list(map(lambda x: "red" if x == 0 else "blue", training.iloc[:,13]))
# pd.plotting.scatter_matrix(training.iloc[:,feature_selection], c=cols, figsize=(12, 12));

In [ ]:
np.random.seed(37)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=3)]

for clf in classifiers:
    clf.fit(x_train, y_train)
    score = clf.score(x_test, y_test)
    print("{} test accuracy: {}".format(clf.__class__.__name__, score))

In [ ]:
pipelines = [Pipeline([('scaler', StandardScaler()), ('clf', clone(clf))]) for clf in classifiers]

for pipeline in pipelines:
    pipeline.fit(x_train, y_train)
    score = pipeline.score(x_test, y_test)
    print("Scaled {} test accuracy: {}".format(pipeline['clf'].__class__.__name__, score))

In [ ]:
rng = np.random.RandomState(0)
clf = RandomForestClassifier(bootstrap=True, max_samples=166, n_estimators=100, random_state=rng)
clf.fit(x_train, y_train)
score = clf.score(x_test, y_test)
print("Test accuracy: {}".format(score))

In [ ]:
clf = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 1000],
    'max_samples': [0.2, 0.9],
    'max_features': [0.1, 0.5],
    'bootstrap': [True, False]
    }

search = GridSearchCV(clf, param_grid, cv=5, verbose=1)

search.fit(x_train, y_train)
score = search.score(x_test, y_test)
print("Best CV score: {} using {}".format(search.best_score_, search.best_params_))
print("Test accuracy: {}".format(score))

In [ ]:
clf = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [0, 20],
    'weights': ["uniform", "distance", "None"],
    'algorithm': ["ball_tree", "kd_tree", "brute", "auto"],
    }

search = GridSearchCV(clf, param_grid, cv=5, verbose=1)

search.fit(x_train, y_train)
score = search.score(x_test, y_test)
print("Best CV score: {} using {}".format(search.best_score_, search.best_params_))
print("Test accuracy: {}".format(score))

In [ ]:
# pipelines = Pipeline([('scaler', StandardScaler()), ('clf', clone(clf))])

clf = KNeighborsClassifier(algorithm= 'ball_tree', n_neighbors=20, weights= 'uniform')
pipelines = Pipeline([('scaler', StandardScaler()), ('clf', clone(clf))])
pipeline.fit(x_train, y_train)
score = pipeline.score(x_test, y_test)
print("Scaled {} test accuracy: {}".format(pipeline['clf'].__class__.__name__, score))

In [ ]:
logitmodel = LogisticRegression(max_iter=10,tol=1,C=0.001)
logitmodel.fit(x_train, y_train)
score = logitmodel.score(x_test, y_test)
print('{0:.4f}'.format(score))